In [1]:
%load_ext autoreload
%autoreload 2

In [30]:
from os.path import join, exists
from os import makedirs
import pickle

import tensorflow as tf
import numpy as np
from PIL import Image

from utils import reset
from yolophem import models

In [21]:
!ls  evaluation

yolophem_A


In [73]:
checkpoint_path = './experiments/yolophem_B_small/parameters'
evaluation_path = './evaluation/yolophem_B/validation'
images_path = './datasets/wider_face/WIDER_val/images/'


CONFIG = models.config_small
MODEL = models.adapted
PARAMS = {} #{ 'feature_size': 512 }

In [74]:
# FOR TEST DATA
with open('./datasets/wider_face/wider_face_split/wider_face_test_filelist.txt') as f:
    lines = f.readlines()

In [75]:
# FOR VALIDATION DATA
meta = pickle.load(open('./datasets/wider_face/metadata.pickle', 'rb'))
lines, _ = meta['val']
lines = [l[37:] for l in lines]

In [76]:
num_lines = len(lines)

In [77]:
reset()
[X, y], [predictions, confidences], loss = MODEL(CONFIG, **PARAMS)

saver = tf.train.Saver()

config = tf.ConfigProto()
config.gpu_options.allow_growth = True

Tensor("conv1/Relu:0", shape=(?, 53, 53, 256), dtype=float32)
Tensor("primaryCaps/outputs/mul:0", shape=(?, 25, 25, 16, 8), dtype=float32)
Tensor("convCaps1/routing/outputs:0", shape=(?, 11, 11, 8, 12), dtype=float32)
Tensor("convCaps2/routing/outputs:0", shape=(?, 9, 9, 3, 16), dtype=float32)
Tensor("convCaps3/routing/outputs:0", shape=(?, 7, 7, 3, 16), dtype=float32)


In [ ]:
with tf.Session(config=config) as sess:
    saver.restore(sess, checkpoint_path)
    
    for it, line in enumerate(lines):
        print('{}/{}'.format(it, num_lines))
        
        image = Image.open(join(images_path, line.strip()))
        width, height = image.size
    
        # Affine Warp
        image_reshaped = image.resize((CONFIG['image_size'], CONFIG['image_size']))
        image_np = np.array(image_reshaped)
        image.close()

        # Standardize
        image_np = (image_np - 255) / 255

        preds, confs = sess.run(
            [predictions, confidences], 
            feed_dict={
                X: [image_np],
                y: [[0,0,0,0,0]],
            }
        )

        preds = np.clip(preds, 0, None).reshape(-1, 4)
        confs = np.clip(confs, 0, 1).reshape(-1)

        # Only report predictions with a confidence larger than .1 percent
        idx = confs > 0.001

        preds = preds[idx]
        confs = confs[idx]

        # Rescale to input resolution
        width_factor = width / CONFIG['image_size']
        height_factor = height / CONFIG['image_size']

        preds = preds * np.array([width_factor, height_factor, width_factor, height_factor])

        # Write to file
        directory, filename = line.split('/')
        name, _ = filename.split('.')

        target_path = join(evaluation_path, directory)

        if not exists(target_path):
            makedirs(target_path)

        with open(join(target_path, name + '.txt'), 'w') as f:
            f.write(name + '\n')
            f.write(str(len(confs)) + '\n')

            for box in np.c_[preds, confs]:
                f.write('{:.0f} {:.0f} {:.0f} {:.0f} {:.3f}\n'.format(*box))


INFO:tensorflow:Restoring parameters from ./experiments/yolophem_B_small/parameters
0/3226
1/3226
2/3226
3/3226
4/3226
5/3226
6/3226
7/3226
8/3226
9/3226
10/3226
11/3226
12/3226
13/3226
14/3226
15/3226
16/3226
17/3226
18/3226
19/3226
20/3226
21/3226
22/3226
23/3226
24/3226
25/3226
26/3226
27/3226
28/3226
29/3226
30/3226
31/3226
32/3226
33/3226
34/3226
35/3226
36/3226
37/3226
38/3226
39/3226
40/3226
41/3226
42/3226
43/3226
44/3226
45/3226
46/3226
47/3226
48/3226
49/3226
50/3226
51/3226
52/3226
53/3226
54/3226
55/3226
56/3226
57/3226
58/3226
59/3226
60/3226
61/3226
62/3226
63/3226
64/3226
65/3226
66/3226
67/3226
68/3226
69/3226
70/3226
71/3226
72/3226
73/3226
74/3226
75/3226
76/3226
77/3226
78/3226
79/3226
80/3226
81/3226
82/3226
83/3226
84/3226
85/3226
86/3226
87/3226
88/3226
89/3226
90/3226
91/3226
92/3226
93/3226
94/3226
95/3226
96/3226
97/3226
98/3226
99/3226
100/3226
101/3226
102/3226
103/3226
104/3226
105/3226
106/3226
107/3226
108/3226
109/3226
110/3226
111/3226
112/3226
113/3226


915/3226
916/3226
917/3226
918/3226
919/3226
920/3226
921/3226
922/3226
923/3226
924/3226
925/3226
926/3226
927/3226
928/3226
929/3226
930/3226
931/3226
932/3226
933/3226
934/3226
935/3226
936/3226
937/3226
938/3226
939/3226
940/3226
941/3226
942/3226
943/3226
944/3226
945/3226
946/3226
947/3226
948/3226
949/3226
950/3226
951/3226
952/3226
953/3226
954/3226
955/3226
956/3226
957/3226
958/3226
959/3226
960/3226
961/3226
962/3226
963/3226
964/3226
965/3226
966/3226
967/3226
968/3226
969/3226
970/3226
971/3226
972/3226
973/3226
974/3226
975/3226
976/3226
977/3226
978/3226
979/3226
980/3226
981/3226
982/3226
983/3226
984/3226
985/3226
986/3226
987/3226
988/3226
989/3226
990/3226
991/3226
992/3226
993/3226
994/3226
995/3226
996/3226
997/3226
998/3226
999/3226
1000/3226
1001/3226
1002/3226
1003/3226
1004/3226
1005/3226
1006/3226
1007/3226
1008/3226
1009/3226
1010/3226
1011/3226
1012/3226
1013/3226
1014/3226
1015/3226
1016/3226
1017/3226
1018/3226
1019/3226
1020/3226
1021/3226
1022/3226
1023/

1744/3226
1745/3226
1746/3226
1747/3226
1748/3226
1749/3226
1750/3226
1751/3226
1752/3226
1753/3226
1754/3226
1755/3226
1756/3226
1757/3226
1758/3226
1759/3226
1760/3226
1761/3226
1762/3226
1763/3226
1764/3226
1765/3226
1766/3226
1767/3226
1768/3226
1769/3226
1770/3226
1771/3226
1772/3226
1773/3226
1774/3226
1775/3226
1776/3226
1777/3226
1778/3226
1779/3226
1780/3226
1781/3226
1782/3226
1783/3226
1784/3226
1785/3226
1786/3226
1787/3226
1788/3226
1789/3226
1790/3226
1791/3226
1792/3226
1793/3226
1794/3226
1795/3226
1796/3226
1797/3226
1798/3226
1799/3226
1800/3226
1801/3226
1802/3226
1803/3226
1804/3226
1805/3226
1806/3226
1807/3226
1808/3226
1809/3226
1810/3226
1811/3226
1812/3226
1813/3226
1814/3226
1815/3226
1816/3226
1817/3226
1818/3226
1819/3226
1820/3226
1821/3226
1822/3226
1823/3226
1824/3226
1825/3226
1826/3226
1827/3226
1828/3226
1829/3226
1830/3226
1831/3226
1832/3226
1833/3226
1834/3226
1835/3226
1836/3226
1837/3226
1838/3226
1839/3226
1840/3226
1841/3226
1842/3226
1843/3226


2564/3226
2565/3226
2566/3226
2567/3226
2568/3226
2569/3226
2570/3226
2571/3226
2572/3226
2573/3226


In [5]:
!ls evaluation/test/

0--Parade	      28--Sports_Fan	       47--Matador_Bullfighter
1--Handshaking	      29--Students_Schoolkids  48--Parachutist_Paratrooper
10--People_Marching   3--Riot		       49--Greeting
11--Meeting	      30--Surgeons	       5--Car_Accident
12--Group	      31--Waiter_Waitress      50--Celebration_Or_Party
13--Interview	      32--Worker_Laborer       51--Dresses
14--Traffic	      33--Running	       52--Photographers
15--Stock_Market      34--Baseball	       53--Raid
16--Award_Ceremony    35--Basketball	       54--Rescue
17--Ceremony	      36--Football	       55--Sports_Coach_Trainer
18--Concerts	      37--Soccer	       56--Voter
19--Couple	      38--Tennis	       57--Angler
2--Demonstration      39--Ice_Skating	       58--Hockey
20--Family_Group      4--Dancing	       59--people--driving--car
21--Festival	      40--Gymnastics	       6--Funeral
22--Picnic	      41--Swimming	       61--Street_Battle
23--Shoppers	      42--Car_Racing	       7--Cheering
24--Soldier_Firing    43--Row_Boat	  

In [189]:
!cat evaluation/test/0--Parade/0_Parade_marchingband_1_737.txt

0_Parade_marchingband_1_737
4
717 235 196 760 0.001
992 372 195 499 0.002
1031 427 477 677 0.002
1216 383 207 466 0.002


In [175]:
for p in np.c_[preds_scaled, confs]:
    print('{:.0f} {:.0f} {:.0f} {:.0f} {:.3f}'.format(*p))

717 235 196 760 0.001
992 372 195 499 0.002
1031 427 477 677 0.002
1216 383 207 466 0.002


In [176]:
len(confs)

4

In [168]:
np.c_[preds_scaled, confs]

array([[7.16969825e+02, 2.35454917e+02, 1.96478516e+02, 7.60181170e+02,
        1.03238598e-03],
       [9.91750215e+02, 3.72485456e+02, 1.95262525e+02, 4.98570599e+02,
        1.88540900e-03],
       [1.03138073e+03, 4.26597048e+02, 4.77154422e+02, 6.76614335e+02,
        2.43243575e-03],
       [1.21584977e+03, 3.82920724e+02, 2.07378691e+02, 4.65549024e+02,
        2.14420538e-03]])

In [106]:
confs_clipped.shape

(1, 147)

In [47]:
directory, filename, name

('0--Parade', '0_Parade_marchingband_1_737.jpg', '0_Parade_marchingband_1_737')

In [60]:
im = Image.open(join(images_path, line))

In [71]:
# Standardize
im = (im - 255) / 255

In [72]:
im.shape

(109, 109, 3)

In [34]:
name + '.txt'

'0_Parade_marchingband_1_737.txt'

In [40]:
exists(evaluation_path)

False

In [42]:
makedirs?

In [37]:
s.replace?